In [ ]:
# Dependencies
# %matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
import time
from datetime import datetime, timedelta, date
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)
pd.set_option('display.max_columns', None)

from scipy.interpolate import CubicSpline


In [ ]:
#diable warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#import North America data and split into Canada and US
na_data = pd.read_csv("na_data.csv")
na_data['Time'] = pd.to_datetime(na_data['Time'])
CND = na_data[(na_data["Country"]=="Canada")]
USA = na_data[(na_data["Country"]=="United States")]

In [ ]:
#add time difference column
CND['time_diff'] = CND['Time'].diff()
USA['time_diff'] = USA['Time'].diff()

In [ ]:
def rolling_in_the_deep(dslic_interpolated):
    '''
    This function takes in a dataframe and returns a dataframe with rolling averages and rates
    '''
    all_data = dslic_interpolated
    all_data = all_data.sort_values(by=['Level','Region', 'Country', 'State/Province','Time']).reset_index()
    all_data = all_data.drop(columns=["index"])
    base_columns = [
        'Level', 'Region','Country','Census Region','State/Province','Abbreviation','FIPS',
        'Time', 'Date', 'Status',
        'Cases Daily', 'Total Cases',
        'Deaths Daily', 'Total Deaths',
        'Tests Daily', 'Total Tests',
        'Population', 'Population 100K',
        'Country Population', 'Country Population 100K',"Country Share",
        'Region Population', 'Region Population 100K',"Region Share",
        'World Population', 'World Population 100K',"World Share",
        'Accessed']
    all_data = all_data[base_columns]
    all_data['Time'] = all_data['Time'].astype(str)
    all_min = all_data["Time"].min()
    all_max = all_data["Time"].max()
    all_data.head()
    #print(all_data['State/Province'].unique())

    grouping_cols = ["Level","Region","Country","State/Province"]
    base_cols = ["Cases","Tests","Deaths"]
    calc_cols = []
    for col in base_cols:
        all_data[col + " Daily"] = all_data[col + " Daily"].fillna(0).astype(int)
        all_data[col + " Weekly"] = all_data[col + " Daily"].rolling(7,min_periods=7).sum().reset_index(drop=True)
        all_data[col + " Daily Rate"] = all_data[col + " Daily"]/all_data["Population 100K"]
        all_data[col + " Weekly Rate"] = all_data[col + " Weekly"]/all_data["Population 100K"]
        all_data[col + " Total"] = all_data.groupby(grouping_cols)[col + " Daily"].cumsum().reset_index(drop=True)
        rolling_col = all_data.groupby(grouping_cols, as_index=False)[col + " Daily"].rolling(7,min_periods=7).mean().reset_index(drop=True)
        all_data[col + " Daily 7D Rolling"] = rolling_col[col + " Daily"]    
        all_data[col + " Daily Rate 7D Rolling"] = all_data.groupby(grouping_cols, as_index=False)["Cases Daily Rate"].rolling(7,min_periods=7).mean().reset_index(drop=True)["Cases Daily Rate"]
        all_data["Total " + col + " Rate"] = all_data[col + " Total"]/all_data["Population 100K"]
        base_order = [col + ' Daily', col + ' Daily 7D Rolling']
        if col!="Vaccinations":
            base_order += ["Total " + col]
        base_order += [col + " Total", col + ' Daily Rate', col + ' Daily Rate 7D Rolling', "Total " + col + " Rate"]
        calc_cols = calc_cols + base_order

    all_data["Speed Daily"] = all_data["Cases Daily 7D Rolling"]/all_data["Population 100K"]
    all_data["Speed Weekly"] = all_data["Cases Weekly"]/all_data["Population 100K"]
    all_data["Positivity 7D Rolling"] = all_data["Cases Daily 7D Rolling"]/all_data["Tests Daily 7D Rolling"]
    all_data["Positivity 7D Rolling"] = all_data["Positivity 7D Rolling"].apply(lambda x: np.nan if x == np.inf else x)
    all_data["Positivity Weekly"] = all_data["Cases Weekly"]/all_data["Tests Weekly"]
    all_data["Positivity Weekly"] = all_data["Positivity Weekly"].apply(lambda x: np.nan if x == np.inf else x)
    all_data["Acceleration Daily"] = all_data.groupby(grouping_cols, as_index=False)["Speed Daily"].diff().reset_index(drop=True)
    all_data["Acceleration Weekly"] = all_data.groupby(grouping_cols, as_index=False)["Speed Weekly"].diff(7).reset_index(drop=True)
    all_data["Jerk Daily"] = all_data.groupby(grouping_cols, as_index=False)["Acceleration Daily"].diff().reset_index(drop=True)
    all_data["Jerk Weekly"] = all_data.groupby(grouping_cols, as_index=False)["Acceleration Weekly"].diff(7).reset_index(drop=True)
    all_data["Jounce Daily"] = all_data.groupby(grouping_cols, as_index=False)["Jerk Daily"].diff().reset_index(drop=True)
    all_data["MM-DD-YYYY"] = all_data["Date"].apply(lambda x: datetime.strptime(x, '%m/%d/%Y').strftime('%m-%d-%Y'))
    all_data["MM-DD-YYYY"] = all_data["MM-DD-YYYY"].astype(str)
    all_data["DD-MM-YYYY"] = all_data["Date"].apply(lambda x: datetime.strptime(x, '%m/%d/%Y').strftime('%d-%m-%Y'))
    all_data["DD-MM-YYYY"] = all_data["DD-MM-YYYY"].astype(str)
    all_data["Week"] = all_data["Date"].apply(
        lambda x:
        str(datetime.strptime(x, '%m/%d/%Y').isocalendar()[0]) +
        " W" + str(datetime.strptime(x, '%m/%d/%Y').isocalendar()[1])
    )
    all_data["First Day of Week"] = all_data["Date"].apply(
        lambda x: datetime.strptime(x, '%m/%d/%Y') - timedelta(days=datetime.strptime(x, '%m/%d/%Y').weekday())
    )
    all_data["Last Day of Week"] = all_data["First Day of Week"].apply(lambda x: x + timedelta(days=6))
    def shortDate(day):
        shortMonth = datetime.strftime(day,'%m').lstrip('0')
        shortDay  = datetime.strftime(day,'%d').lstrip('0')
        shortYear = datetime.strftime(day,'%y')
        return shortMonth + "/" + shortDay + "/" + shortYear
    all_data["Week Date Range"] = all_data.apply(
        lambda x: shortDate(x["First Day of Week"]) + " - " + shortDate(x["Last Day of Week"]),
        axis=1
    )
    all_data["Day Count"] = all_data.groupby(["Region","Country","State/Province","Date"]).cumcount()

    header_cols = [
        'Level', 'Region','Country','Census Region','State/Province','Abbreviation','FIPS',
        'Time', 'Date', 'Day Count', 'MM-DD-YYYY', 'DD-MM-YYYY', 'Week', 'First Day of Week', 'Last Day of Week', 'Week Date Range',
        'Status'
    ]
    calc_cols = calc_cols + [
        "Positivity 7D Rolling", "Positivity Weekly",
        "Speed Daily", "Speed Weekly",
        "Acceleration Daily","Acceleration Weekly",
        "Jerk Daily", "Jerk Weekly"
    ]
    other_cols = [
        #'People Vaccinated', 'People Fully Vaccinated',
    #    'Recovered Daily', 'Total Recovered',
    #    'Active Daily', 'Total Active',
        'Population', 'Population 100K',
        'Country Population', 'Country Population 100K',"Country Share",
        'Region Population', 'Region Population 100K',"Region Share",
        'World Population', 'World Population 100K',"World Share",
        'Accessed'#,'Stringency'
    ]
    all_cols = header_cols + calc_cols + other_cols
    all_data = all_data[all_cols].copy()
    last_monday = date(year=2021,month=1,day=18)
    all_min = all_data["Time"].min()
    all_max = all_data["Time"].max()
    all_data.head()
    return(all_data)

In [ ]:
#This is the version that deaths daily are not divided by 7,
#the next block is the version that deaths daily are divided by 7,
#use accordingly
cnd_new = pd.DataFrame()

takina = ['Alberta', 'British Columbia', 'Manitoba', 'New Brunswick',
       'Newfoundland and Labrador', 'Northwest Territories',
       'Nova Scotia', 'Nunavut', 'Ontario', 'Prince Edward Island',
       'Quebec', 'Repatriated Travellers', 'Saskatchewan', 'Yukon']

for pro in takina:
    # get the daily data for the country
    prod = CND[CND['State/Province'] == pro]
    prod['Cases Daily'] = round(prod['Cases Daily']/7)
    #prod['Deaths Daily'] = round(prod['Deaths Daily']/7)
    prod['Tests Daily'] = round(prod['Tests Daily']/7)
    
    prod = prod.reset_index()
    prod = prod.set_index('Time')  # set new index to 'Time' column
    
    dslic = prod.resample('D').ffill()
    dslic['orderc'] = dslic['Cases Daily'].rolling(window=7, min_periods=1).mean()
    dslic['orderd'] = dslic['Deaths Daily'].rolling(window=7, min_periods=1).mean()
    dslic['ordert'] = dslic['Tests Daily'].rolling(window=7, min_periods=1).mean()    
    print(dslic['State/Province'].unique())
    
    csc = CubicSpline(dslic.index.astype(int), dslic['orderc'])
    csd = CubicSpline(dslic.index.astype(int), dslic['orderd'])
    cst = CubicSpline(dslic.index.astype(int), dslic['ordert'])
    
    # generate new x values for the spline
    new_dates = pd.date_range(dslic.index.min(), dslic.index.max(), freq='D')
    
    # evaluate the spline at the new x values
    dslic_interpolated = dslic.reindex(new_dates)
    dslic_interpolated['Cases Daily'] = csc(new_dates.astype(int))
    dslic_interpolated['Deaths Daily'] = csd(new_dates.astype(int))
    dslic_interpolated['Tests Daily'] = cst(new_dates.astype(int))
    
    dslic_interpolated = dslic_interpolated.reset_index()
    dslic_interpolated = dslic_interpolated.rename(columns={'level_0': 'Time'})
      
    all_data = rolling_in_the_deep(dslic_interpolated)
    cnd_new = cnd_new.append(all_data, ignore_index=True)

In [ ]:
cnd_new.to_csv("cnd_new_fx_0516.csv")

In [ ]:
#This is the version that deaths daily are divided by 7
cnd_new = pd.DataFrame()

takina = ['Alberta', 'British Columbia', 'Manitoba', 'New Brunswick',
       'Newfoundland and Labrador', 'Northwest Territories',
       'Nova Scotia', 'Nunavut', 'Ontario', 'Prince Edward Island',
       'Quebec', 'Repatriated Travellers', 'Saskatchewan', 'Yukon']

for pro in takina:
    # get the daily data for the country
    prod = CND[CND['State/Province'] == pro]
    prod['Cases Daily'] = round(prod['Cases Daily']/7)
    prod['Deaths Daily'] = round(prod['Deaths Daily']/7)
    prod['Tests Daily'] = round(prod['Tests Daily']/7)
    
    prod = prod.reset_index()
    prod = prod.set_index('Time')  # set new index to 'Time' column
    
    dslic = prod.resample('D').ffill()
    dslic['orderc'] = dslic['Cases Daily'].rolling(window=7, min_periods=1).mean()
    dslic['orderd'] = dslic['Deaths Daily'].rolling(window=7, min_periods=1).mean()
    dslic['ordert'] = dslic['Tests Daily'].rolling(window=7, min_periods=1).mean()    
    print(dslic['State/Province'].unique())
    
    csc = CubicSpline(dslic.index.astype(int), dslic['orderc'])
    csd = CubicSpline(dslic.index.astype(int), dslic['orderd'])
    cst = CubicSpline(dslic.index.astype(int), dslic['ordert'])
    
    # generate new x values for the spline
    new_dates = pd.date_range(dslic.index.min(), dslic.index.max(), freq='D')
    
    # evaluate the spline at the new x values
    dslic_interpolated = dslic.reindex(new_dates)
    dslic_interpolated['Cases Daily'] = csc(new_dates.astype(int))
    dslic_interpolated['Deaths Daily'] = csd(new_dates.astype(int))
    dslic_interpolated['Tests Daily'] = cst(new_dates.astype(int))
    
    dslic_interpolated = dslic_interpolated.reset_index()
    dslic_interpolated = dslic_interpolated.rename(columns={'level_0': 'Time'})
      
    all_data = rolling_in_the_deep(dslic_interpolated)
    cnd_new = cnd_new.append(all_data, ignore_index=True)

In [ ]:
cnd_new.to_csv("cnd_new_d7_fx_0516.csv")

In [ ]:
#Now we calculate the usa data, also using cubic splines
#USA data switched to weekly since October, 2022
#you can see the state names to make sure your code is running
usa_new = pd.DataFrame()

for pro in USA['State/Province'].unique():
    # get the daily data for the country
    slic = USA[USA['State/Province'] == pro]
    slic_1 = pd.DataFrame()
    slic_not_1 = pd.DataFrame()

    # Loop through each row of the original dataframe
    for index, row in slic.iterrows():
        my_string = str(row["time_diff"])[0]
        numeric_string = ''.join([char for char in my_string if char.isdigit()])
        if len(numeric_string) != 0:
            my_int = int(numeric_string)
        else:
            my_int = 1

        if my_int != 1 and my_int != 0:
            # Add the row to the dataframe where my_int != 1
            row['Cases Daily'] = round(row['Cases Daily']/my_int)
            row['Deaths Daily'] = round(row['Deaths Daily']/my_int)
            row['Tests Daily'] = round(row['Tests Daily']/my_int)
            slic_not_1 = slic_not_1.append(row)
        elif my_int == 0:
            continue
        else:
            slic_1 = slic_1.append(row)
    
    slic_1_zx = rolling_in_the_deep(slic_1)
    last_row = slic_1.tail(1).copy()
    slic_not_1 = slic_not_1.reset_index(drop=True)
    slic_not_1 = pd.concat([last_row, slic_not_1], ignore_index=True)
    slic_not_1 = slic_not_1.reset_index()
    slic_not_1 = slic_not_1.set_index('Time')
    dslic = slic_not_1.resample('D').ffill()

    dslic['orderc'] = dslic['Cases Daily'].rolling(window=7, min_periods=1).mean()
    dslic['orderd'] = dslic['Deaths Daily'].rolling(window=7, min_periods=1).mean()
    dslic['ordert'] = dslic['Tests Daily'].rolling(window=7, min_periods=1).mean()    
    print(dslic['State/Province'].unique())
    
    csc = CubicSpline(dslic.index.astype(int), dslic['orderc'])
    csd = CubicSpline(dslic.index.astype(int), dslic['orderd'])
    cst = CubicSpline(dslic.index.astype(int), dslic['ordert'])
    
    # generate new x values for the spline
    new_dates = pd.date_range(dslic.index.min(), dslic.index.max(), freq='D')
    
    # evaluate the spline at the new x values
    dslic_interpolated = dslic.reindex(new_dates)
    dslic_interpolated['Cases Daily'] = csc(new_dates.astype(int))
    dslic_interpolated['Deaths Daily'] = csd(new_dates.astype(int))
    dslic_interpolated['Tests Daily'] = cst(new_dates.astype(int))
    
    dslic_interpolated = dslic_interpolated.reset_index()
    dslic_interpolated = dslic_interpolated.rename(columns={'level_0': 'Time'})
    
    slic_not_1_zx = rolling_in_the_deep(dslic_interpolated)
    slic_not_1_zx = slic_not_1_zx.drop(slic_not_1_zx.index[0])

    state_d = pd.concat([slic_1_zx, slic_not_1_zx], ignore_index=True)
    usa_new = usa_new.append(state_d, ignore_index=True)

In [ ]:
usa_new.to_csv("usa_new_fx_0516.csv")

Now you have the data ready to be used in step 3, below is a piece of viz code to help you understand the data better.

In [ ]:
nmd = usa_new[usa_new['State/Province'] == 'New Mexico']
fig, ax = plt.subplots(figsize=(15, 6))
ax.plot(nmd['Time'], nmd['Cases Daily Rate 7D Rolling'])

# set the size of the plot

# add axis labels and title
plt.xlabel('Date')
plt.ylabel('Cases Daily 7')
plt.title('Daily Cases (7-day rolling average)')

# display the plot
plt.show()